In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats
import random
from matplotlib.patches import Ellipse
import math

pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_csv('tentamen_data.csv')
#make datetime object, such it can be substrected
df['purchase'] = pd.to_datetime(df['purchase'])
df['redeem'] = pd.to_datetime(df['redeem'])




df['red-pur']=df['redeem'] - df['purchase']
df['t1']='2014-04-07'
df['t1'] = pd.to_datetime(df['t1'])
df['t1-pur']=df['t1']-df['purchase'] 
df_r = df
df_un = df

#remove every not NaT row
df_r = df_r[pd.notnull(df_r['red-pur'])]
#remove every NaT row
df_un = df_un[pd.isnull(df_un['red-pur'])]


#removes days from diff
# df_r['diff'] = df_r['diff'].apply(lambda x: x / np.timedelta64(1,'D'))
df_r['red-pur'] = df_r['red-pur'].astype('timedelta64[D]').astype(int)
df_r['t1-pur'] = df_r['t1-pur'].astype('timedelta64[D]').astype(int)


#model for gift card usage
def Pt(p, q, drp):
    return p* np.exp(-q*drp-(p/q)*(1-np.exp(-q*(drp))))

#normalization factor
def Norm(p, q, dtp):
    return 1- np.exp(-(p/q)*(1-np.exp(-q*(dtp))))

#log-likelihood
def LLH(p, q, drp, dtp):
    
    L = - 2 * np.log(Pt(p, q, drp)/Norm(p, q, dtp))
    return L

def calculate_LLH(pq):
    p, q = pq
    for i in df_r.index:
#         print('red-pur: ')
#         print(df_r["red-pur"][i])
#         print('t1-pur: ')
#         print(df_r["t1-pur"][i])
        LLH(p, q, df_r["red-pur"][i], df_r["t1-pur"][i])       
     

def mini():
    min_res = minimize(calculate_LLH, np.array([0.1, 0.1]), method = "SLSQP", bounds = [(0,1),(None,None)])
    [p, q] = min_res.x
    return min_res.x
    
min_res = mini()

# print(df_r.set_index("red-pur"))
print(min_res)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'